In [ ]:
pip install openai && pip install python-dotenv

In [30]:
from openai import OpenAI
from dotenv import dotenv_values

In [31]:
# Set the Open API key from the .env file
config = dotenv_values(".env")
client = OpenAI(
  api_key=config["OPEN_API_KEY"]
)

In [35]:
# Create a completion model using gpt-3.5-turbo
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    { "role": "user", "content": "The dog says " } # role = "user" means you or who is chatting or who is asking the question
  ],
)

In [39]:
print(completion)

ChatCompletion(id='chatcmpl-9vBVSY2lkCyj5gxp7fyc3P71pmn4B', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='"woof"', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1723415954, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=4, prompt_tokens=11, total_tokens=15))


### Analysis of ChatCompletion API
The result from the prompt "The dog says " was "woof" from the gpt-3.5-turbo (defaulted to gpt-3.5-turbo-0125) model. If you check your OpenAI API usage, you'll see the billing usage of the API.

As of today (08/11/2024), the pricing for gpt-3.5-turbo-0125 is $0.50 / 1M tokens for input and $1.50 / 1M tokens for output.

See billing usage in the website https://platform.openai.com/organization/usage:

![billing](../images/api-usage-gpt-3.5-turbo.png)